Passpunkte

| 0  | 1  | 2  | 3  |
|----|----|----|----|
| 4  | 5  | 6  | 7  |
| 8  | 9 | 10 | 11 |
| 12 | 13 | 14 | 15 |

In [32]:
passpunkte = [
    [0, 0, 0],
    [1, 0, 0],
    [2, 0, 0],
    [3, 0, 0],
    [0, 1, 0],
    [1, 1, 0],
    [2, 1, 0],
    [3, 1, 0],
    [0, 2, 0],
    [1, 2, 0],
    [2, 2, 0],
    [3, 2, 0],
    [0, 3, 0],
    [1, 3, 0],
    [2, 3, 0],
    [3, 3, 0]
]


In [33]:
# kamera, bildnr, passpunkt, bildkoordinatenx, y
messungen = [
    [0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0],
    [0, 0, 2, 2, 0],
    [0, 0, 4, 0, 1],
    [0, 0, 5, 1, 1],
    [0, 0, 6, 2, 1],
    [0, 0, 8, 0, 2],
    [0, 0, 9, 1, 2],
    [0, 0, 10, 2, 2],
    [0, 1, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 1, 3, 2, 0],
    [0, 1, 5, 0, 1],
    [0, 1, 6, 1, 1],
    [0, 1, 7, 2, 1],
    [0, 1, 9, 0, 2],
    [0, 1, 10, 1, 2],
    [0, 1, 11, 2, 2],
    [0, 2, 4, 0, 0],
    [0, 2, 5, 1, 0],
    [0, 2, 6, 2, 0],
    [0, 2, 8, 0, 1],
    [0, 2, 9, 1, 1],
    [0, 2, 10, 2, 1],
    [0, 2, 12, 0, 2],
    [0, 2, 13, 1, 2],
    [0, 2, 14, 2, 2],
    [0, 3, 5, 0, 0],
    [0, 3, 6, 1, 0],
    [0, 3, 7, 2, 0],
    [0, 3, 9, 0, 1],
    [0, 3, 10, 1, 1],
    [0, 3, 11, 2, 1],
    [0, 3, 13, 0, 2],
    [0, 3, 14, 1, 2],
    [0, 3, 15, 2, 2],
    [0, 4, 0, 0, 0],
    [0, 4, 1, 1, 0],
    [0, 4, 2, 2, 0],
    [0, 4, 3, 3, 0],
    [0, 4, 4, 0, 1],
    [0, 4, 5, 1, 1],
    [0, 4, 6, 2, 1],
    [0, 4, 7, 3, 1],
    [0, 4, 8, 0, 2],
    [0, 4, 9, 1, 2],
    [0, 4, 10, 2, 2],
    [0, 4, 11, 3, 2],
    [0, 4, 12, 0, 3],
    [0, 4, 13, 1, 3],
    [0, 4, 14, 2, 3],
    [0, 4, 15, 3, 3]
]


In [34]:
# x,y,z, height, width
bild = [
    [1, 1, 1, 2,2],
    [2, 1, 1, 2, 2],
    [1, 2, 1, 2, 2],
    [2, 2, 1, 2, 2],
    [1.5, 1.5, 1.5, 3, 3]
]


In [35]:
import bz2
import os
import time
import urllib

import numpy as np
from scipy.optimize import least_squares
from scipy.sparse import lil_matrix


In [36]:
camera_indices = np.array([i[0] for i in messungen])
image_indices = np.array([i[1] for i in messungen])
point_indices = np.array([i[2] for i in messungen])
points_2d = np.array([[i[3]/bild[i[1]][3]-0.5, i[4]/bild[i[1]][4]-0.5]
                     for i in messungen])
points_3d = np.array(passpunkte)
image_params = np.array([[0, 1, 0, i[0], i[1], i[2]] for i in bild])
camera_params = np.array([[0.5, 0.5, 0.5]])


In [37]:
n_cameras = camera_params.shape[0]
n_images = image_params.shape[0]
n_points = points_3d.shape[0]

n = n_cameras * 3 + 6 * n_images + 3 * n_points
m = 2 * points_2d.shape[0]

print("n_cameras: {}".format(n_cameras))
print("n_images: {}".format(n_images))
print("n_points: {}".format(n_points))
print("Total number of parameters: {}".format(n))
print("Total number of residuals: {}".format(m))

n_cameras: 1
n_images: 5
n_points: 16
Total number of parameters: 81
Total number of residuals: 104


In [38]:
def rotate(points, rot_vecs):
    """Rotate points by given rotation vectors.
    
    Rodrigues' rotation formula is used.
    """
    theta = np.linalg.norm(rot_vecs, axis=1)[:, np.newaxis]
    with np.errstate(invalid='ignore'):
        v = rot_vecs / theta
        v = np.nan_to_num(v)
    dot = np.sum(points * v, axis=1)[:, np.newaxis]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)

    return cos_theta * points + sin_theta * np.cross(v, points) + dot * (1 - cos_theta) * v


In [39]:
def project(points, camera_params, image_params):
    """Convert 3-D points to 2-D by projecting onto images."""
    points_proj = rotate(points, image_params[:, :3])
    points_proj += image_params[:, 3:6]
    points_proj = -points_proj[:, :2] / points_proj[:, 2, np.newaxis]
    f = camera_params[:, 0]
    k1 = camera_params[:, 1]
    k2 = camera_params[:, 2 ]
    n = np.sum(points_proj**2, axis=1)
    r = 1 + k1 * n + k2 * n**2
    points_proj *= (r * f)[:, np.newaxis]
    return points_proj


In [40]:
def fun(params, n_cameras, n_images, n_points, camera_indices, images_indices, point_indices, points_2d):
    """Compute residuals.

    `params` contains camera parameters and 3-D coordinates.
    """
    camera_params = params[:n_cameras * 3].reshape((n_cameras, 3))
    image_params = params[n_cameras * 3: n_cameras *
                          3+n_images*6].reshape((n_images, 6))
    points_3d = params[n_cameras*3+n_images*6:].reshape((n_points, 3))
    points_proj = project(
        points_3d[point_indices], camera_params[camera_indices], image_params[image_indices])
    return (points_proj - points_2d).ravel()


In [41]:
def bundle_adjustment_sparsity(n_cameras, n_images, n_points, camera_indices, image_indices, point_indices):
    m = image_indices.size * 2
    n = n_cameras * 3 + n_images * 6 + n_points * 3
    A = lil_matrix((m, n), dtype=int)

    i = np.arange(camera_indices.size)

    for s in range(3):
        A[2 * i, camera_indices * 3 + s] = 1
        A[2 * i + 1, camera_indices * 3 + s] = 1

    for s in range(6):
        A[2 * i, n_cameras * 3 + image_indices * 6 + s] = 1
        A[2 * i + 1, n_cameras * 3+ image_indices * 6 + s] = 1

    for s in range(3):
        A[2 * i, n_cameras * 3 + n_images * 6 + point_indices * 3 + s] = 1
        A[2 * i + 1, n_cameras * 3 + n_images * 6 + point_indices * 3 + s] = 1

    return A


In [42]:
x0 = np.hstack((camera_params.ravel(), image_params.ravel(), points_3d.ravel()))
f0 = fun(x0, n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d)
x0.reshape(27,3)

array([[0.5, 0.5, 0.5],
       [0. , 1. , 0. ],
       [1. , 1. , 1. ],
       [0. , 1. , 0. ],
       [2. , 1. , 1. ],
       [0. , 1. , 0. ],
       [1. , 2. , 1. ],
       [0. , 1. , 0. ],
       [2. , 2. , 1. ],
       [0. , 1. , 0. ],
       [1.5, 1.5, 1.5],
       [0. , 0. , 0. ],
       [1. , 0. , 0. ],
       [2. , 0. , 0. ],
       [3. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 1. , 0. ],
       [2. , 1. , 0. ],
       [3. , 1. , 0. ],
       [0. , 2. , 0. ],
       [1. , 2. , 0. ],
       [2. , 2. , 0. ],
       [3. , 2. , 0. ],
       [0. , 3. , 0. ],
       [1. , 3. , 0. ],
       [2. , 3. , 0. ],
       [3. , 3. , 0. ]])

In [43]:

A = bundle_adjustment_sparsity(
    n_cameras, n_images, n_points, camera_indices, image_indices,  point_indices)
t0 = time.time()
res = least_squares(fun, x0, jac_sparsity=A, verbose=2, x_scale='jac', ftol=1e-4, method='trf',
                    args=(n_cameras, n_images, n_points, camera_indices, image_indices, point_indices, points_2d))
t1 = time.time()


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.8832e+14                                    1.71e+16    
       1              3         1.1316e+14      7.52e+13       1.06e+01       1.01e+16    
       2              4         3.9845e+13      7.33e+13       2.35e+01       3.43e+15    
       3              5         4.4343e+12      3.54e+13       5.32e+01       3.91e+14    
       4              6         3.4764e+11      4.09e+12       5.29e+01       4.08e+13    
       5              7         4.4616e+10      3.03e+11       1.14e+01       4.67e+12    
       6              8         1.0544e+10      3.41e+10       1.38e+01       1.75e+12    
       7              9         7.2914e+08      9.81e+09       3.97e+00       1.76e+11    
       8             10         5.8294e+08      1.46e+08       4.44e+01       9.18e+10    
       9             11         5.8188e+07      5.25e+08       2.80e+00       2.99e+09    

In [44]:
x0
np.array(res.x.round(1)).reshape(27,3)

array([[  0. , -20.2,   0.1],
       [  0. ,   1. ,  -0. ],
       [ -0.5,   1.1,   1. ],
       [  0. ,   1. ,   0. ],
       [ -0.5,   1.1,   1. ],
       [  0. ,   1. ,   0. ],
       [ -0.6,   1.4,   1. ],
       [  0. ,   1. ,   0. ],
       [  0.6,   1.2,   1. ],
       [  0. ,   1. ,   0. ],
       [ -0.3,   1. ,   1.5],
       [ -9. , -17.3, -12.4],
       [  1.1,  -1.1,  -0.1],
       [  1.9,  -2.4,  -0.1],
       [ -5.2,  13.2, -12.8],
       [-14.5, -11.9, -19.2],
       [  1. ,   1.4,  -0. ],
       [  1.9,  -1.6,   0. ],
       [-21.2,  19.3, -42.7],
       [-12.5,   8.1, -17.1],
       [  1. ,   2. ,  -0. ],
       [  2. ,  -0.2,   0. ],
       [-21. , -20.6, -41.5],
       [ -7.2,  12.3, -10.7],
       [  1. ,   2.3,   0. ],
       [  2. ,   1.1,  -0. ],
       [-16.2, -36.1, -31.7]])